# Installation

In [1]:
# !pip install guidance transformers

# Load a Model

In [2]:
import guidance

# Or any model from LlamaCpp, OpenAI, Azure AI, etc...
lm_orig = guidance.models.Transformers(
    # "microsoft/phi-2", 
    "microsoft/Phi-3-mini-4k-instruct",
    trust_remote_code=True,
    device_map="mps",
    temperature=0, do_sample=True
)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
question = """\
30 students are in a class. 1/5 of them are 12 years old, 1/3 are 13 years old.
1/10 of them are 11 years old.
How many of them are not 11, 12, or 13 years old?
"""
# The Correct answer is 11. 
# 3 are 11, 6 are 12, 10 are 13. 
# 30-3-6-10 = 11

print(question)

30 students are in a class. 1/5 of them are 12 years old, 1/3 are 13 years old.
1/10 of them are 11 years old.
How many of them are not 11, 12, or 13 years old?



## Basic Prompting Alone

In [5]:
from guidance import gen
lm = lm_orig

lm += "QUESTION: " + question # Load the LM with the question
lm += "\n\nANSWER: " + gen("answer", max_tokens=20) # Generate an answer

## Better Prompting

In [6]:
from guidance import gen, select
lm = lm_orig

lm += f"""
Help me with this math question.
QUESTION: {question}
Let's pick: should we write out our STEPS first or can we directly ANSWER the question?
"""
lm += select(["STEPS", "ANSWER"], name='choice') # Let the LM pick what it wants to do!
if lm['choice'] == 'STEPS':
    lm += ":\n" + gen("steps", max_tokens=300, stop="\n\n")

# If the LM picks "ANSWER", it will generate the answer directly
lm += ":\n" + gen("answer", max_tokens=100)

In [7]:
# Let's get an extractable summary
lm += "\nTherefore, in summary, the final answer is: " + gen("final", regex="[0-9]+")

In [8]:
print(lm["final"])

11


In [15]:
# Put it in a reusable function
from guidance import user, assistant

@guidance
def solve_math(lm, question):
    lm += f"""
    Help me with this math question.
    QUESTION: {question}
    Let's pick: should we write out our STEPS first or can we directly ANSWER the question?
    """
    lm += select(["STEPS", "ANSWER"], name='choice') # Let the LM pick what it wants to do!
    if lm['choice'] == 'STEPS':
        lm += ":\n" + gen("steps", max_tokens=300, stop="\n\n")

    # If the LM picks "ANSWER", it will generate the answer directly
    lm += ":\n" + gen("answer", max_tokens=200)
    lm += "\nTherefore, in summary, the final answer is: " + gen("final", regex='[0-9]+')
    return lm